In [1]:
import boto3
import cufflinks as cf
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import warnings

from io import StringIO
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
cf.go_offline()
init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')

In [3]:
def get_secret():
    secret_name = "s3_data_analysis"
    region_name = "us-east-1"

    # Create a Secrets Manager client with default credentials
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        # Retrieve the secret value from Secrets Manager
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    # Extract the secret string (which is a JSON object)
    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string as JSON
    secrets = json.loads(secret)
    
    # Retrieve AWS credentials from the secrets
    aws_access_key_id = secrets['aws_access_key_id']
    aws_secret_access_key = secrets['aws_secret_access_key']
    
    # Now use the retrieved credentials to create the S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

    return s3_client

s3_client = get_secret()

In [ ]:
def read_csv_from_s3(bucket_name, key, s3_client=None):
    
    if s3_client is None:
        s3_client = boto3.client('s3')
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=key)
        csv_content = response['Body'].read().decode('utf-8')
        data = StringIO(csv_content)
        df = pd.read_csv(data)
        return df
    except Exception as e:
         print(f"Error reading CSV from S3: {e}")
         return None

bucket_name = 'nk.data.analysis'
key = 'Bitcoin_Analysis/btcusd_1-min_data.csv'

df = read_csv_from_s3(bucket_name, key, s3_client)

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

df = df.sort_index(ascending=False).reset_index()

df.drop('index', axis=1, inplace=True)

df.set_index('Timestamp', inplace=True)

In [ ]:
df_daily = df.resample('D').last().dropna()

df_daily = df_daily.sort_index(ascending=False)

In [ ]:
plt.figure(figsize = (20,12))
           
for index, col in enumerate(['Open', 'High', 'Low', 'Close'], 1):
    plt.subplot(2,2,index)
    plt.plot(df_daily.index, df_daily[col])
    plt.title(col)

In [ ]:
bitcoin_sample = df_daily[0:50]

In [ ]:
trace = go.Candlestick(
    x=bitcoin_sample.index,
    high=bitcoin_sample['High'],
    low=bitcoin_sample['Low'],
    open=bitcoin_sample['Open'],
    close=bitcoin_sample['Close'],
    increasing_line=dict(width=2),
    decreasing_line=dict(width=2)
)

fig = go.Figure(data=[trace])

fig.update_layout(
    title='Bitcoin Historical Prices',
    xaxis=dict(
        title='Date',
        tickformat='%b %d, %Y',
        tickangle=-45
    ),
    yaxis=dict(
        title='Price',
        gridcolor='lightgray',
        zerolinecolor='black'
    ),
    hovermode='x unified',
    width=1000,  # Increase figure size
    height=600,
    xaxis_rangeslider_visible=False
)

fig.show()


In [ ]:
df_daily['Close'].plot()

In [ ]:
plt.figure(figsize=(20,6))

plt.subplot(1,2,2)
np.log1p(df_daily['Close']).plot()
plt.title('Log Scaling')
plt.yscale('log')

In [ ]:
df_daily['Close'].resample('Y').mean().plot()

In [ ]:
df_daily['Close'].resample('Q').mean().plot()

In [ ]:
df_daily['Close'].resample('M').mean().plot()

In [ ]:
df_daily['close_price_pct_change'] = df_daily['Close'].pct_change()*100

In [ ]:
df_daily['close_price_pct_change'].plot()

In [ ]:
df_daily['close_price_pct_change'].iplot()